<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [Import bibliothèques et fichiers utiles](#0)
- [Récupération commentaires API Yelp](#1)
- [Traitement](#2)
- [Création vocabulaire grâce aux données](#3)
- [Transformation des mots en indices](#4)
- [Création modèles](#5)
- [Pipeline](#6)
- [Test des modèles sur notre jeu de données](#7)
- [Détermination du seuil selon les données observées](#8)

<a name="0"></a>
# Import bibliothèques et fichiers utiles

#####Import des bibliothèques nécessaires pour la tokenization, stemming, lemmatization, des bibliothèques nécessaires pour le deep learning et autres (numpy, pandas...)

In [16]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [17]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
pip install --pre gql[all]

In [19]:
import requests
import json

<a name="1"></a>
# Récupération commentaires API Yelp

#####On récupère les commentaires qui nous permettront de  tester notre pipeline via l'API Yelp (n'en transmet que 3)

In [20]:
api_key='MI6JGndpuW3Kyc9cIjj5of9AhZUKET4JlGdRsiBnGZjL2ofbChjG0ksWdjXrHvdRN3cHosvu6bkDcvdOUCc5A_m-IAciq5kpjSqAxt5euSYLzB0q0vvWqmuyrI86YHYx'
#Clé API nécessaire pour l'utilisation de l'API Yelp (propre à chaque utilisateur)
headers = {'Authorization': 'Bearer %s' % api_key}

url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)

dataset=json.loads(req.text)["reviews"] #On récupère les commentaires fournis par l'API Yelp
x_test=[]
for i in range(len(dataset)):
  x_test.append(dataset[i]["text"])
x_test2=x_test.copy() #On a créé 2 variables pour tester nos 2 modèles
print(x_test)
print(x_test2)

["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']
["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']


<a name="2"></a>
# Traitement

#####Mise en minuscule de tous les mots, tokenization, suppression des stopwords, stemming et lemmatization, la fonction prend en entrée la donnée à traiter et renvoie la donnée après traitement

In [21]:
def processing(data): #Fonction qui effectue tous le traitement nécessaire (minuscules, tokenization, suppression des stopwords, stemming et lemmatization)
  if (isinstance(data,str)): #Pour permettre de tester nos modèles sur une phrase brute et non pas un tablea, on vérifie le type de données que l'on a en entrée
#Si la donnée est une chaîne de caractère on va effectuer le traitement sur celle-ci
    #MINUSCULES
    data=data.lower()
    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    data=tokenizer.tokenize(data)

    #STOPWORDS
    data=[w for w in data if not w in list(nltk.corpus.stopwords.words('english'))]
    #STEMMING
    stemmer=LancasterStemmer()
    data=[stemmer.stem(w) for w in data]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    data=[Word_Lemmatizer.lemmatize(w) for w in data]

  elif (isinstance(data,list)): #Si la donnée est une liste (pour tester nos modèles sur une liste de phrases)
    for i in range(len(data)): #On parcourt juste chaque élément de cette liste à chaque étape du traitement pour faire le même traitement que précédemment, mais
    #sur chacune des phrases indépendamment
      data[i]=data[i].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      data[i]=tokenizer.tokenize(data[i])

    #STOPWORDS
    for i in range(len(data)):
      data[i]=[w for w in data[i] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      data[i]=[stemmer.stem(w) for w in data[i]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      data[i]=[Word_Lemmatizer.lemmatize(w) for w in data[i]]

  else : #Enfin, cette condition nous permet de traiter nos données pour l'entraînement de nos modèles, la forme de ces données était légèrement différente car
  #plusieurs dimensions : liste de 2 listes (positif/négatif)
  #Le traitement est le même mais on a 2 boucles for imbriquées
    #MINUSCULES
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=data[i][j].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=tokenizer.tokenize(data[i][j])

    #STOPWORDS
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[stemmer.stem(w) for w in data[i][j]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]

  return data #On renvoie la donnée traitée

<a name="3"></a>
# Création du vocabulaire grâce aux données

#####On crée notre vocabulaire nécessaire en tant que dictionnaire, qui contient comme clés l'ensemble des mots de notre dataset et comme valeurs leurs indices. Il contient une clé-valeur spécifique pour le padding

In [22]:
from collections import defaultdict
def defineVocab(data): #Fonction qui définit le vocabulaire à partir des données passées en entrée
  v=set() #On crée un set qui est très utile pour déterminer le vocabulaire car il est rempli sans avoir de doublons 
  for i in range(len(data)):
    for j in range(len(data[i])):
      for h in range(len(data[i][j])): #On parcourt tous les MOTS de notre jeu de données (ce qui explique les 3 boucles for imbriquées)
        v.add(data[i][j][h]) #Comme expliqué précédemment, on ajoute le mot au set, mais celui-ci ne va être ajouté que s'il n'est pas déjà présent

  vocab = defaultdict(lambda: 0) #On initialise le dictionnaire de telle manière que pour toute valeur non présente en tant que clé dans celui-ci, la valeur attribuée sera ici 0
  #Autrement dit, lorsqu'un mot ne sera pas présent dans le dictionnaire (lors de la prédiction), son indice correspondant sera "0"
  vocab['<PAD>'] = 1 #Comme on va effectuer du padding, on définit une valeur pour du padding : la valeur 1

  j=2 #On commence donc l'initialisation d'indices à partir de la valeur 2
  for i in v: #Ici, on va donc remplir le dictionnaire (vocabulaire), pour chacun des mots du set avec un indice quelconque
    vocab[i]=j
    j+=1
  return vocab

<a name="4"></a>
# Transformation des mots en indices

#####A l'aide du vocabulaire déterminé précédemment, on transforme les mots en leurs indices correspondants. Si le mot est dans le vocabulaire, on le change en son indice, s'il est nécessaire de faire du padding, il prendra son indice également, en revanche, s'il n'est pas dans le vocabulaire, il prendra la valeur "0"

In [23]:
def transformWords(data,vocab): #Fonction qui va transformer les mots en leur indice correspondant dans le vocabulaire
  matrixData=[]
  maxLen=0 #variable qui va stocker la taille de la plus grande phrase du jeu de données
  if (isinstance(data,list)): #De la même manière que pour la fonction de traitement, on a adapté selon la forme de la donnée
  #Ici, on teste si la donnée en entrée est une liste, cette condition est respectée lors de la prédiction, que celle-ci soit effectuée pour une phrase brute 
  #(car elle est transformée en liste de mots après le traitement) ou pour une liste de phrases
    if (len(data)>1): #On distingue justement ces 2 cas, si c'est une liste de phrases, la taille de la liste sera supérieure à 1 
      for i in range(len(data)):
        tmp=[] #On initialise une liste tmp qui va contenir les indices des mots transformés, et ce pour chacune des phrases
        for j in data[i]:
          if (j not in vocab.keys()): #Comme expliqué précédemment, si le mot de la prédiction n'est pas dans le vocabulaire, il prend la valeur 0, ici on ne procède pas comme
          #décrit précédemment car le vocabulaire ici a été chargé depuis le fichier CSV et n'est donc plus un defaultdict
            tmp.append(0)
          else: #Sinon, on ajoute l'indice correspondant au mot
            tmp.append(vocab[j])
        matrixData.append(tmp) #On ajoute la liste d'indices au fur et à mesure dans la liste finale
    else: #Sinon elle sera de 1
      for i in data:
        if (i not in vocab.keys()):
          matrixData.append(0)
        else:
          matrixData.append(vocab[i])
  else: #L'autre condition est respectée uniquement lors de l'entraînement car c'est un numpy array
    for i in range(len(data)): #Le traitement est le même, à l'exception qu'ici on doit faire du padding donc on doit d'abord récupéré la taille maximale
      for j in range(len(data[i])):
        if (maxLen < len(data[i][j])):
          maxLen= len(data[i][j])

    for i in range(len(data)):
      for j in range(len(data[i])):
        tmp=[]
        for k in range(maxLen):
          if (k >= len(data[i][j])): #Si l'on a atteint la fin de la phrase mais qu'elle doit subir du padding, on lui ajoute les indices correspondants jusqu'à avoir
          #la taille requise
            tmp.append(vocab['<PAD>'])
          else:
            tmp.append(vocab[data[i][j][k]])
        matrixData.append(tmp)
  return matrixData

<a name="5"></a>
# Création modèles

#####On crée nos 2 modèles pour notre baseline : les 2 sont basés sur du RNN, le 1er étant du LSTM, l'autre du simple RNN

In [24]:
def buildModelLSTM(vocab): #On construit notre modèle LSTM
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1)) #On ajoute une couche d'embedding en prenant en entrée la taille du vocabulaire 
  #(+1 sinon l'entraînement ne s'exécute pas )
  model.add(tf.keras.layers.LSTM(128,activation="relu")) #On ajoute une couche LSTM avec la fonction d'activation relu
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))  #On ajoute une couche finale avec la fonction d'activation sigmoid
  return model
  #Pour plus de détails : voir le rapport

In [25]:
def buildModelSimpleRNN(vocab): #On construit notre modèle Simple RNN, identique au précédent mais avec une couche Simple RNN plutôt que LSTM pour la couche cachée
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.SimpleRNN(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [26]:
def fitModel(data,model,negSize,posSize): #negSize et posSize = taille des données prises en compte pour le fit, respectivement des commentaires négatifs et positifs
# Ex : si on crée le modèle en lui donnant 20 commentaires négatifs et 20 positifs, negSize=20 et posSize=20
  y_train=np.concatenate([np.zeros((negSize,1)),np.ones((posSize,1))]) #On définit les valeurs attendues en en prenant le bon nombre par rapport aux valeurs d'entrée
  #on définit cette array avec des 0 pour les commentaires négatifs et avec des 1 pour les commentaires positifs
  model.compile(loss='binary_crossentropy',optimizer="adam") #On compile le modèle avec la fonction de perte binary crossentrop et l'optimizer adam
  model.fit(data,y_train,epochs=50) #On fait l'entraînement avec les valeurs d'entrée et nos valeurs attendues définies plus haut, entraînement de 50 epochs

<a name="6"></a>
# Pipeline

#####Pipeline qui permet de prendre en entrée uniquement la donnée dont on veut prédire le sentiment (phrase brute ou liste de phrases), et qui nous ressort le sentiment prédit par le modèle. Il y a 2 pipelines : une pour le 1er modèle, une pour le 2ème.
#####On réutilise toutes les fonctions définies précédemment, on crée ou on charge notre modèle selon s'il est déjà existant ou non, et on fait notre prédiction après avoir fait le traitement de notre donnée.

In [27]:
import os
def final_LSTM(sentence): #Pipeline pour LSTM
  threshold=0.4 #Seuil déterminé grâce à la dernière fonction (voir rapport pour plus de détails)
  model_file="LSTM_Model" #Nom de notre modèle (va être utilisé pour sauvegarder et charger notre modèle)
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)): #OIn regarde si le modèle n'existe pas déjà, si ce n'est pas le cas, on entre dans cette boucle
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None) #On récupère les données du dataset
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]]) #On ne prend que le texte des commentaires, et on prend ici 2000 données (1000 positives, 1000 négatives)
    x_train=processing(x_train) #On effectue le traitement grâce à notre fonction processing
    vocab=defineVocab(x_train) #On détermine le vocabulaire de notre dataset grâce à notre fonction defineVocab
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"]) #On crée une dataframe qui contient les clés et valeurs correspondantes de notre vocabulaire
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab") #On exporte notre vocabulaire en CSV
    x_train=transformWords(x_train,vocab) #On transforme nos mots en indices grâce à notre fonction transformWords
    x_train=np.array(x_train) #On transforme notre x_train en array pour qu'il ait la forme souhaitée pour notre modèle
    model=buildModelLSTM(vocab) #On construit notre modèle LSTM
    fitModel(x_train,model,1000,1000) #On l'entraîne
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file) #On le sauvegarde
  else: #Si le modèle existe déjà, on se contente de le charger
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"] #On récupère également le vocabulaire que l'on retransforme en dictionnaire
    #avec comme clé les mots et comme valeurs les indices correspondants
  # return model #Décommenter cette ligne pour utilise utiliser la fonction de détermination de seuil
  sentence=processing(sentence) #On fait le traitement sur la donnée saisie en entrée de notre pipeline
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1): #On doit distinguer si la donnée que l'on veut prédire est une phrase brute ou une liste de phrase, ici c'est le cas
  #où celle-ci est une phrase brute
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold): #Si la prédiction est inférieure au seuil défini plus haut, alors la phrase est négative, sinon elle est positive
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else: #On fait pareil ici mais pour la liste de phrases, commentTypes devient donc une liste
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




In [28]:
import os
def final_SimpleRNN(sentence): #Ici on fait la même chose que précédemment mais pour notre autre modèle (simple RNN)
  threshold=0.2 #Il n'y a que le seuil qui change, le nom de notre modèle, et on appelle la fonction de création du modèle Simple RNN et non LSTM
  model_file="SimpleRNN_Model"
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelSimpleRNN(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
  # return model
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




<a name="7"></a>
# Test des modèles sur notre jeu de données

#####Simple appel de nos pipelines avec les données récupérées de l'API Yelp pour tester nos modèles

In [29]:
final_LSTM(x_test) #On teste notre pipeline avec les 2 modèles (sur les 2 cellules)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

In [30]:
final_SimpleRNN(x_test2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

<a name="8"></a>
# Détermination du seuil selon les données observées

#####Détermination du seuil à utiliser pour que notre modèle soit le plus précis possible dans ses prédictions, et ce à l'aide du calcul du F1 score pour une liste prédéfinie de seuils

In [31]:
from sklearn.metrics import f1_score
def defineThreshold(datas,model): #Fonction qui va déterminer le seuil à utiliser pour être le plus précis possible avec le modèle fourni sur les données fournies
  y_train=np.concatenate([np.zeros((1000,1)),np.ones((1000,1))])
  seuils=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] #on définit l'ensemble des seuils qui vont être testés
  F1_score=[] #Va contenir l'ensemble des F1 scores, pour plus d'information sur les F1 scores, consultez le rapport
  predict=model.predict(datas) #On récupère l'ensemble des prédictions du modèle sur les données fournies
  for seuil in seuils:
    predictions=[] #Va contenir l'ensemble des valeurs finales de prédictions (0 ou 1 selon la nature du commentaire et le seuil)
    for i in range(len(datas)): #On parcourt l'ensemble des valeurs renvoyées par le réseau de neurones
      if (predict[i]<seuil): #Si la prédiction est inférieure au seuil, alors on estime que le commentaire est négatif (on ajoute donc un "0")
        predictions.append(0)
      else: #Sinon, positif (un "1")
        predictions.append(1)
    F1_score.append(f1_score(y_train,predictions)) #On se sert des résultats attendus et des prédictions à l'aide du seuil pour déterminer le F1 score et donc la précision
    #pour chacun des seuils
  print(F1_score)
  seuil=seuils[F1_score.index(max(F1_score))] #Le seuil renvoyé finalement correspond au maximum des F1 score (celui avec lequel le modèle est le plus précis)
  print(seuil)
  return seuil #On renvoie ce seuil

In [32]:
# defineThreshold(np.array(dat),Simple_RNN) #Permet de tester notre fonction de seuil, à décommenter pour tester (mais attention bien décommenter les lignes correspondantes,
#dans les pipelines)